In [4]:
import tensorflow as tf
import numpy as np
from model import *
from metrics import *
from dataset import *


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
image_size = 256

In [ ]:
train_gen = data_generator(data,image_size, batch_size, num_classes, flag='train')
valid_gen = data_generator(data,image_size, batch_size, num_classes, flag='valid')
test_gen = data_generator(data,image_size, batch_size, num_classes, flag='test')

In [ ]:
x = tf.placeholder("float", shape=[256*256]) # 병원 데이터는 dimension이 256*256
y = tf.placeholder("float", shape=[256*256]) 

correct_prediction = tf.equal(tf.argmax())

In [ ]:
image = Input(shape=(image_size, image_size, 1))
y_onehot = test_gen

In [ ]:
for GRADIENT_STEP in np.linspace(.01, .1, 10):
    adv_image = GRADIENT_STEP * signed_grad + image
    pred2 = sess.run(y_conv, feed_dict={x:adv_image, keep_prob:1.0})
    label2 = np.argmax(pred2)
    # pred = Predicted probs, pred2 = predicted probs adversarial

In [5]:
np.linspace(.01, .1, 10)

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ])

In [19]:
base = 32
scale = 2
num_classes = 4
data = "chd"
image_size = 256
batch_size = 1
data_gen_args_x = dict(rescale=1/255.0,
                       width_shift_range=0.2,
                       height_shift_range=0.2,
                       zoom_range=0.2,
                      )
data_gen_args_y = dict(width_shift_range=0.2,
                       height_shift_range=0.2,
                       zoom_range=0.2,
                      )
seed = 103
x_datagen = ImageDataGenerator(**data_gen_args_x)
y_datagen = ImageDataGenerator(**data_gen_args_y)


x_gen = x_datagen.flow_from_directory(
            "./input/"+data+"/x_train/",
            class_mode = None,
            color_mode = "grayscale",
            target_size = (image_size, image_size),
            batch_size = batch_size,
            shuffle = True,
            seed = seed)
y_gen = y_datagen.flow_from_directory(
            "./input/"+data+"/y_train/",
            class_mode = None,
            color_mode = "grayscale",
            target_size = (image_size, image_size),
            batch_size = batch_size,
            shuffle = True,
            seed = seed)
data_gen = zip(x_gen, y_gen) 

Found 56750 images belonging to 1 classes.
Found 56750 images belonging to 1 classes.


In [20]:
x_gen[1]

array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]]], dtype=float32)

In [21]:
np.shape(x_gen[1])

(1, 256, 256, 1)

In [22]:
np.shape(y_gen[1])

(1, 256, 256, 1)

In [47]:
np.max(y_gen[277][0]) # Checking maximum value of an image -> seems to be 256 bit  / but not in whole numbers (demicals)

228.2446

In [58]:
x_sample = x_gen[0]
#x_sample = np.squeeze(x_sample)
y_sample = y_gen[0]
#y_sample = np.squeeze(y_sample)

In [59]:
np.max(y_sample)

229.12427

In [60]:
sample_gen = zip(x_sample, y_sample)

In [61]:
np.shape(sample_gen)

()

In [63]:
import random
import cv2
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import keras
import argparse

from model import *
from metrics import *
from dataset import *
from PIL import Image, ImageEnhance
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras import backend as K

In [65]:
input_img = Input(shape=(image_size, image_size, 1))
model = U_Net(input_img, base, scale, num_classes)

In [66]:
model

In [67]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 256, 256, 32) 320         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_13 (BatchNo (None, 256, 256, 32) 128         conv2d_15[0][0]                  
__________________________________________________________________________________________________
activation_13 (Activation)      (None, 256, 256, 32) 0           batch_normalization_13[0][0]     
__________________________________________________________________________________________________
conv2d_16 

In [ ]:
pred = sess.run(y_conv)